In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join('..', '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from cyberattacks_detection.models import RBFNN, ELM, ModelWrapper, min_max_scale, reverse_min_max_scale
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import re

c:\Users\zgore\anaconda3\envs\cyber-attk-p39\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:103: FutureWarning: Model's `predict` method contains invalid parameters: {'X'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
c:\Users\zgore\anaconda3\envs\cyber-attk-p39\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:134: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [4]:
import mlflow
from mlflow.models import infer_signature

In [5]:
pd.set_option('display.max_colwidth', None)

# Przygotowanie danych

In [6]:
data_path = "../../data/four_tanks/"

In [7]:
# # dane bez zakłóceń
# df_train = pd.read_csv(data_path+"result_ol_without_noise_uczące_v3.csv", sep=';', index_col=0)
# df_val = pd.read_csv(data_path+"result_ol_without_noise_walidacyjne_v3.csv", sep=';', index_col=0)
# df_test = pd.read_csv(data_path+"result_ol_without_noise_testowe_v3.csv", sep=';', index_col=0)
# dataset_name = "bez zakłóceń"

In [8]:
# dane z zakłóceniami
df_train = pd.read_csv(data_path+"result_ol_with_noise_uczące_v5.csv", sep=';', index_col=0)
df_val = pd.read_csv(data_path+"result_ol_with_noise_walidacyjne_v5.csv", sep=';', index_col=0)
df_test = pd.read_csv(data_path+"result_ol_with_noise_testowe_v5.csv", sep=';', index_col=0)
dataset_name = "z zakłóceniami"

In [9]:
df_train.head()

,q_A [cm^3/s],q_B [cm^3/s],q_d1 [cm^3/s],q_d2 [cm^3/s],q_d3 [cm^3/s],q_d4 [cm^3/s],x1 [cm],x2 [cm],x3 [cm],x4 [cm]
0,452.78,555.56,0.2646,0.2390,0.3064,0.2126,65.000000,66.000000,65.000000,66.000000
1,452.78,555.56,0.0600,0.0853,-0.1379,0.0666,65.000000,66.000000,65.000000,66.000000
2,452.78,555.56,0.1468,-0.0172,0.0172,-0.0115,65.000000,66.000000,65.000000,66.000000
3,452.78,555.56,0.3361,0.0377,-0.0206,-0.1637,65.000000,66.000000,65.000000,66.000000
4,452.78,555.56,0.2801,-0.1816,0.2048,-0.0656,65.320426,65.975004,65.017598,65.828939


In [10]:
def transform_column_name(df):
    df.columns = [col[0] for col in df.columns.str.split()]
    return df

In [11]:
df_train = transform_column_name(df_train)
df_val = transform_column_name(df_val)
df_test = transform_column_name(df_test)

In [12]:
qa_max = round(3260000/3600, 2)
qa_min=0
qb_max = round(4000000/3600)
qb_min=0
x_max = [136,
         136,
         130,
         130]
x_min = [20,
         20,
         20,
         20]

In [13]:
def add_delay(df, col, steps=1):    
    df[col+f'(k-{steps})'] = df[col].shift(steps)
    return df

In [14]:
def add_delay_to_all_cols(df_train, df_val, df_test):
    cols_delay = ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']
    for col in cols_delay:
        for step in range(1, 5):
            df_train = add_delay(df_train, col, step)
            df_val = add_delay(df_val, col, step)
            df_test = add_delay(df_test, col, step)
    return df_train, df_val, df_test

In [15]:
df_train, df_val, df_test = add_delay_to_all_cols(df_train, df_val, df_test)

In [16]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)
df_test.dropna(inplace=True)

In [17]:
print(f"Wymiar danych uczących: {df_train.shape}")
print(f"Wymiar danych walidacyjnych: {df_val.shape}")
print(f"Wymiar danych testowych: {df_test.shape}")

Wymiar danych uczących: (2995, 34)
Wymiar danych walidacyjnych: (995, 34)
Wymiar danych testowych: (995, 34)


# Modelowanie

In [18]:
mlflow.set_tracking_uri("http://localhost:5000")

In [19]:
# model_class_dict = {"LR": LinearRegression, "ELM": ELM, "RBF": RBFNN}
# model_class_str = list(model_class_dict.keys())[0]

In [20]:
y_names = ['x1', 'x2', 'x3', 'x4']

In [21]:
model_type_dict = {'statespace': 'w przestrzeni stanu', 'io': 'wejście-wyjście'}

In [22]:
def normalized_x_error(RMSE, y_name):
    pattern = r'x(\d+)'
    idx = int(re.findall(pattern, y_name)[0]) - 1
    return RMSE/(x_max[idx] - x_min[idx])

In [23]:
def calc_NRMSE(df_train, df_val, df_test, y_name, y_pred_name):
    RMSE_train = metrics.root_mean_squared_error(df_train[y_name], df_train[y_pred_name])
    RMSE_val = metrics.root_mean_squared_error(df_val[y_name], df_val[y_pred_name])
    RMSE_test = metrics.root_mean_squared_error(df_test[y_name], df_test[y_pred_name])
    
    NRMSE_train = round(normalized_x_error(RMSE_train, y_name), 4)
    NRMSE_val = round(normalized_x_error(RMSE_val, y_name), 4)
    NRMSE_test = round(normalized_x_error(RMSE_test, y_name), 4)

    return NRMSE_train, NRMSE_val, NRMSE_test

In [24]:
def calc_RMSE(df_train, df_val, df_test, y_name, y_pred_name):
    RMSE_train = round(metrics.root_mean_squared_error(df_train[y_name], df_train[y_pred_name]), 4)
    RMSE_val = round(metrics.root_mean_squared_error(df_val[y_name], df_val[y_pred_name]), 4)
    RMSE_test = round(metrics.root_mean_squared_error(df_test[y_name], df_test[y_pred_name]), 4)

    return RMSE_train, RMSE_val, RMSE_test

In [25]:
def predict_recursion(df, model, features, y_name, order_dyn):
    y_pred_list = np.empty(len(df))
    start_idx = 0
    if order_dyn == 1:
        y_pred_list[0] = df[y_name].iloc[0]
        start_idx = 1
    elif order_dyn==2:
        y_pred_list[0] = df[y_name].iloc[0]
        y_pred_list[1] = df[y_name].iloc[1]
        start_idx = 2
    elif order_dyn==3:
        y_pred_list[0] = df[y_name].iloc[0]
        y_pred_list[1] = df[y_name].iloc[1]
        y_pred_list[2] = df[y_name].iloc[2]
        start_idx = 3
    elif order_dyn==4:
        y_pred_list[0] = df[y_name].iloc[0]
        y_pred_list[1] = df[y_name].iloc[1]
        y_pred_list[2] = df[y_name].iloc[2]
        y_pred_list[3] = df[y_name].iloc[3]
        start_idx = 4
    for i in range(start_idx, len(df)):
        features_val = df[features].iloc[i].copy()
        # print(f'features_val:\n{features_val}')
        if order_dyn == 1:
            features_val[f'{y_name}(k-1)'] = y_pred_list[i-1]
        if order_dyn == 2:
            features_val[f'{y_name}(k-1)'] = y_pred_list[i-1]
            features_val[f'{y_name}(k-2)'] = y_pred_list[i-2]
        if order_dyn == 3:
            features_val[f'{y_name}(k-1)'] = y_pred_list[i-1]
            features_val[f'{y_name}(k-2)'] = y_pred_list[i-2]
            features_val[f'{y_name}(k-3)'] = y_pred_list[i-3]
        if order_dyn == 4:
            features_val[f'{y_name}(k-1)'] = y_pred_list[i-1]
            features_val[f'{y_name}(k-2)'] = y_pred_list[i-2]
            features_val[f'{y_name}(k-3)'] = y_pred_list[i-3]
            features_val[f'{y_name}(k-4)'] = y_pred_list[i-4]
        # print(f'features_val:\n{features_val}')
        try:
            y_pred_list[i] = model.predict(pd.DataFrame(features_val).T).values.flatten()[0]
        except:
            y_pred_list[i] = model.predict(pd.DataFrame(features_val).T).flatten()[0]
        # print(f'y_pred: {y_pred_list[i]}')
    return y_pred_list

In [ ]:
def train_and_evaluate_model(df_train,
                        df_val,
                        df_test,
                        model_class_str,
                        y_names,
                        features_list,
                        model_type,
                        order_dyn,
                        pred_mode='both',
                        features_min_list=[None, None, None, None],
                        features_max_list=[None, None, None, None],
                        **kwargs):
    # **kwargs = num_hidden_neurons=None, n_centers=None, alpha=None
    for y_name, features, features_min, features_max in zip(y_names,
                                                            features_list,
                                                            features_min_list,
                                                            features_max_list):
        
        print(y_name)

        # mlflow.set_experiment(f"four_tanks_{y_name.replace('x', 'h')}_v2")
        mlflow.set_experiment(f"test_{y_name.replace('x', 'h')}_v2")

        if model_class_str == 'LR':
            model = LinearRegression()
            model.fit(df_train[features], df_train[[y_name]])

        elif model_class_str == 'ELM':
            num_features = len(features)
            num_hidden_neurons = kwargs["num_hidden_neurons"]
            model = ELM(num_features, num_hidden_neurons)
            pattern = r'x(\d+)'
            idx = int(re.findall(pattern, y_name)[0]) - 1
            model.fit(df_train[features],
                      df_train[[y_name]],
                      features_min,
                      features_max,
                      x_min[idx],
                      x_max[idx])
            
        elif model_class_str == 'RBF':
            n_centers = kwargs["n_centers"]
            alpha = kwargs["alpha"]
            model = RBFNN(n_centers, alpha)
            pattern = r'x(\d+)'
            idx = int(re.findall(pattern, y_name)[0]) - 1
            iterations = kwargs["iterations"]
            patience = kwargs["patience"]
            it = model.fit(df_train[features],
                      df_train[[y_name]],
                      features_min,
                      features_max,
                      x_min[idx],
                      x_max[idx],
                      iterations=iterations,
                      X_val=df_val[features],
                      y_val=df_val[[y_name]],
                      patience=patience,
                      metric=metrics.mean_squared_error)
            if it is not None:
                kwargs["iterations_real"] = it
            else:
                kwargs["iterations_real"] = kwargs["iterations"]

        params = {"model_class": model_class_str,
                  "model_type": model_type_dict[model_type],
                  "order_dyn": order_dyn,
                  "y_name": y_name+'(k)',
                  "features": features} | kwargs
        
        pred_mode_tuple = ("both", "not_recursion", "recursion")
        # TODO: sprawdzić jak zadziała w przypadku błędnej wartości
        metrics_not_rec = {}
        metrics_rec = {}
        if pred_mode not in pred_mode_tuple:
            raise ValueError(f"Bad value for recursion_mode agument. Possible values: {pred_mode_tuple}")
        if pred_mode in ("both", "not_recursion"):
            df_train[f'{y_name}_{model_type}_{order_dyn}_pred'] = model.predict(df_train[features])
            df_val[f'{y_name}_{model_type}_{order_dyn}_pred'] = model.predict(df_val[features])
            df_test[f'{y_name}_{model_type}_{order_dyn}_pred'] = model.predict(df_test[features])
            RMSE_train, RMSE_val, RMSE_test = calc_RMSE(df_train,
                                                        df_val,
                                                        df_test,
                                                        y_name,
                                                        f'{y_name}_{model_type}_{order_dyn}_pred')
            metrics_not_rec = {"RMSE_train": RMSE_train, "RMSE_val": RMSE_val, "RMSE_test": RMSE_test}
            NRMSE_train, NRMSE_val, NRMSE_test = calc_NRMSE(df_train,
                                                            df_val,
                                                            df_test,
                                                            y_name,
                                                            f'{y_name}_{model_type}_{order_dyn}_pred')
            metrics_norm_not_rec = {"NRMSE_train": NRMSE_train,
                                    "NRMSE_val": NRMSE_val,
                                    "NRMSE_test": NRMSE_test}
        if pred_mode in ("both", "recursion"):
            df_train[f'{y_name}_{model_type}_{order_dyn}_recursion_pred'] = predict_recursion(df_train, model, features, y_name, order_dyn)
            df_val[f'{y_name}_{model_type}_{order_dyn}_recursion_pred'] = predict_recursion(df_val, model, features, y_name, order_dyn)
            df_test[f'{y_name}_{model_type}_{order_dyn}_recursion_pred'] = predict_recursion(df_test, model, features, y_name, order_dyn)
            RMSE_train, RMSE_val, RMSE_test = calc_RMSE(df_train, df_val, df_test, y_name, f'{y_name}_{model_type}_{order_dyn}_recursion_pred')
            metrics_rec = {"RMSE_train_recursion": RMSE_train, "RMSE_val_recursion": RMSE_val, "RMSE_test_recursion": RMSE_test}
            NRMSE_train, NRMSE_val, NRMSE_test = calc_NRMSE(df_train,
                                                            df_val,
                                                            df_test,
                                                            y_name,
                                                            f'{y_name}_{model_type}_{order_dyn}_recursion_pred')
            metrics_norm_rec = {"NRMSE_train_recursion": NRMSE_train,
                                "NRMSE_val_recursion": NRMSE_val,
                                "NRMSE_test_recursion": NRMSE_test}

        metrics_list = metrics_not_rec | metrics_norm_not_rec | metrics_rec | metrics_norm_rec
        
        with mlflow.start_run():
            mlflow.log_params(params)
            mlflow.log_metrics(metrics_list)

            # Infer the model signature
            signature = infer_signature(df_train[features], df_train[f'{y_name}_{model_type}_{order_dyn}_pred'])
            # Log the model
            if model_class_str == 'LR':
                model_info = mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path=model_class_str,
                signature=signature,
                input_example=df_train[features],
                registered_model_name=f"{model_class_str}_{model_type}_{order_dyn}_{y_name}",
            )
            else:
                model_info = mlflow.pyfunc.log_model(
                    python_model=ModelWrapper(model),
                    artifact_path=model_class_str,
                    signature=signature,
                    input_example=df_train[features],
                    registered_model_name=f"{model_class_str}_{model_type}_{order_dyn}_{y_name}",
                )

    return df_train, df_val, df_test

In [27]:
pred_mode = "both"
model_class_list = ["LR", "ELM", "RBF"]

## LinearRegression

In [28]:
model_class_str = model_class_list[0]
print(model_class_str)

LR


In [ ]:
features_list = [['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)', 'x1(k-1)'],
                 ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x3(k-1)', 'x4(k-1)', 'x2(k-1)'],
                 ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x4(k-1)', 'x3(k-1)'],
                 ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)']]
model_type = 'statespace'
order_dyn = 1
df_train, df_val, df_test = train_and_evaluate_model(df_train, df_val, df_test, model_class_str, y_names, features_list, model_type, order_dyn, pred_mode)

x1


c:\Users\zgore\anaconda3\envs\cyber-attk-p39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Registered model 'LR_statespace_1_x1' already exists. Creating a new version of this model...
2025/06/03 06:05:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_statespace_1_x1, version 15
Created version '15' of model 'LR_statespace_1_x1'.


🏃 View run enchanting-elk-857 at: http://localhost:5000/#/experiments/335176418275410378/runs/88369e7e7d824d8482acdec6848c33c8
🧪 View experiment at: http://localhost:5000/#/experiments/335176418275410378
x2


Registered model 'LR_statespace_1_x2' already exists. Creating a new version of this model...
2025/06/03 06:05:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_statespace_1_x2, version 13
Created version '13' of model 'LR_statespace_1_x2'.


🏃 View run debonair-cat-632 at: http://localhost:5000/#/experiments/801974650724713131/runs/e01c41deeb864982a7b9b658f940f00e
🧪 View experiment at: http://localhost:5000/#/experiments/801974650724713131
x3


Registered model 'LR_statespace_1_x3' already exists. Creating a new version of this model...
2025/06/03 06:05:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_statespace_1_x3, version 11
Created version '11' of model 'LR_statespace_1_x3'.


🏃 View run nebulous-stork-20 at: http://localhost:5000/#/experiments/852635572895522263/runs/0052072cb4cf4910a0916e5c018c7f3e
🧪 View experiment at: http://localhost:5000/#/experiments/852635572895522263
x4


Registered model 'LR_statespace_1_x4' already exists. Creating a new version of this model...
2025/06/03 06:05:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_statespace_1_x4, version 9


🏃 View run sedate-grouse-865 at: http://localhost:5000/#/experiments/793997268641947506/runs/1d12dfd7cca6410a86547eb117cd4729
🧪 View experiment at: http://localhost:5000/#/experiments/793997268641947506


Created version '9' of model 'LR_statespace_1_x4'.


In [ ]:
features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)']]
model_type = 'io'
order_dyn = 1
df_train, df_val, df_test = train_and_evaluate_model(df_train, df_val, df_test, model_class_str, y_names, features_list, model_type, order_dyn, pred_mode)

x1


Registered model 'LR_io_1_x1' already exists. Creating a new version of this model...
2025/06/03 06:06:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_io_1_x1, version 6
Created version '6' of model 'LR_io_1_x1'.


🏃 View run omniscient-asp-8 at: http://localhost:5000/#/experiments/335176418275410378/runs/fd572e1b0b9f40d38d2c8a8edd63a1e8
🧪 View experiment at: http://localhost:5000/#/experiments/335176418275410378
x2


Registered model 'LR_io_1_x2' already exists. Creating a new version of this model...
2025/06/03 06:06:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_io_1_x2, version 6
Created version '6' of model 'LR_io_1_x2'.


🏃 View run gifted-fish-150 at: http://localhost:5000/#/experiments/801974650724713131/runs/cd52be164f75411e98fa6bc5c370b01d
🧪 View experiment at: http://localhost:5000/#/experiments/801974650724713131
x3


Registered model 'LR_io_1_x3' already exists. Creating a new version of this model...
2025/06/03 06:06:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_io_1_x3, version 6
Created version '6' of model 'LR_io_1_x3'.


🏃 View run defiant-carp-45 at: http://localhost:5000/#/experiments/852635572895522263/runs/79da8f87bc1248f4aca8d1c8ce4283b1
🧪 View experiment at: http://localhost:5000/#/experiments/852635572895522263
x4


Registered model 'LR_io_1_x4' already exists. Creating a new version of this model...
2025/06/03 06:06:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR_io_1_x4, version 6


🏃 View run marvelous-grub-346 at: http://localhost:5000/#/experiments/793997268641947506/runs/d6f61b1213644747969d12f8cd45e09b
🧪 View experiment at: http://localhost:5000/#/experiments/793997268641947506


Created version '6' of model 'LR_io_1_x4'.


In [ ]:
features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)']]
model_type = 'io'
order_dyn = 2
df_train, df_val, df_test = train_and_evaluate_model(df_train, df_val, df_test, model_class_str, y_names, features_list, model_type, order_dyn, pred_mode)

In [ ]:
features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x1(k-3)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x2(k-3)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x3(k-3)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x4(k-3)']]
model_type = 'io'
order_dyn = 3
df_train, df_val, df_test = train_and_evaluate_model(df_train, df_val, df_test, model_class_str, y_names, features_list, model_type, order_dyn, pred_mode)

In [ ]:
features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x1(k-3)', 'q_A(k-4)', 'q_B(k-4)', 'x1(k-4)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x2(k-3)', 'q_A(k-4)', 'q_B(k-4)', 'x2(k-4)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x3(k-3)', 'q_A(k-4)', 'q_B(k-4)', 'x3(k-4)'],
 ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x4(k-3)', 'q_A(k-4)', 'q_B(k-4)', 'x4(k-4)']]
model_type = 'io'
order_dyn = 4
df_train, df_val, df_test = train_and_evaluate_model(df_train, df_val, df_test, model_class_str, y_names, features_list, model_type, order_dyn, pred_mode)

## Extreme Learning Machine

In [31]:
model_class_str = model_class_list[1]
print(model_class_str)

ELM


In [32]:
num_hidden_neurons_list = [30, 40, 60, 80, 100, 120]

In [33]:
y_names = ['x1', 'x2', 'x3', 'x4']

In [ ]:
for num_hidden_neurons in num_hidden_neurons_list:
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)', 'x1(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x3(k-1)', 'x4(k-1)', 'x2(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x4(k-1)', 'x3(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[1], x_min[2], x_min[3], x_min[0]],
                        [qa_min, qb_min, x_min[0], x_min[2], x_min[3], x_min[1]],
                        [qa_min, qb_min, x_min[0], x_min[1], x_min[3], x_min[2]],
                        [qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[1], x_max[2], x_max[3], x_max[0]],
                        [qa_max, qb_max, x_max[0], x_max[2], x_max[3], x_max[1]],
                        [qa_max, qb_max, x_max[0], x_max[1], x_max[3], x_max[2]],
                        [qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]])
    model_type = 'statespace'
    order_dyn = 1
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    num_hidden_neurons = num_hidden_neurons)
    
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 1
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    num_hidden_neurons = num_hidden_neurons)
    
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 2
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    num_hidden_neurons = num_hidden_neurons)
    
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x1(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x2(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x3(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x4(k-3)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 3
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    num_hidden_neurons = num_hidden_neurons)

## Radial Base Function Neural Network

In [ ]:
model_class_str = model_class_list[2]
print(model_class_str)

In [ ]:
n_centers_list = [20, 25, 30, 35, 40]
alpha = 0.01

In [ ]:
y_names = ['x1', 'x2', 'x3', 'x4']

In [ ]:
for n_centers in n_centers_list:
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)', 'x1(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x3(k-1)', 'x4(k-1)', 'x2(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x4(k-1)', 'x3(k-1)'],
                    ['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'x2(k-1)', 'x3(k-1)', 'x4(k-1)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[1], x_min[2], x_min[3], x_min[0]],
                        [qa_min, qb_min, x_min[0], x_min[2], x_min[3], x_min[1]],
                        [qa_min, qb_min, x_min[0], x_min[1], x_min[3], x_min[2]],
                        [qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[1], x_max[2], x_max[3], x_max[0]],
                        [qa_max, qb_max, x_max[0], x_max[2], x_max[3], x_max[1]],
                        [qa_max, qb_max, x_max[0], x_max[1], x_max[3], x_max[2]],
                        [qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]])
    model_type = 'statespace'
    order_dyn = 1
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    n_centers=n_centers,
                                                    alpha=alpha,
                                                    iterations=300,
                                                    patience=100)
    
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 1
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    n_centers=n_centers,
                                                    alpha=alpha,
                                                    iterations=300,
                                                    patience=100)
    
    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 2
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    n_centers=n_centers,
                                                    alpha=alpha,
                                                    iterations=300,
                                                    patience=100)

    features_list = [['q_A(k-1)', 'q_B(k-1)', 'x1(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x1(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x1(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x2(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x2(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x2(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x3(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x3(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x3(k-3)'],
    ['q_A(k-1)', 'q_B(k-1)', 'x4(k-1)', 'q_A(k-2)', 'q_B(k-2)', 'x4(k-2)', 'q_A(k-3)', 'q_B(k-3)', 'x4(k-3)']]
    features_min_list = np.array([[qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0], qa_min, qb_min, x_min[0]],
                        [qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1], qa_min, qb_min, x_min[1]],
                        [qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2], qa_min, qb_min, x_min[2]],
                        [qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3], qa_min, qb_min, x_min[3]]])
    features_max_list = np.array([[qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0], qa_max, qb_max, x_max[0]],
                        [qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1], qa_max, qb_max, x_max[1]],
                        [qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2], qa_max, qb_max, x_max[2]],
                        [qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3], qa_max, qb_max, x_max[3]]])
    model_type = 'io'
    order_dyn = 3
    df_train, df_val, df_test = train_and_evaluate_model(df_train,
                                                    df_val,
                                                    df_test,
                                                    model_class_str,
                                                    y_names,
                                                    features_list,
                                                    model_type,
                                                    order_dyn,
                                                    pred_mode,
                                                    features_min_list=features_min_list,
                                                    features_max_list=features_max_list,
                                                    n_centers=n_centers,
                                                    alpha=alpha,
                                                    iterations=300,
                                                    patience=100)